## Introduction

Let's explore the statistics of cards drawn [without replacement](https://en.wikipedia.org/wiki/Simple_random_sample) from a deck an equal number of black and red cards. What I want to characterize is how much card draws behave similarly to the simpler [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution) behavior of coin flips.  We expect that the statistics of a small number of draws from a large deck look a lot like the statistics of independent coin flips. However, it is well known that [card counting](https://en.wikipedia.org/wiki/Card_counting) works for decks of cards, and not for dice/coins. So there are differences to characterize.

A really fun example of how card draws differ from coin flips is the *risk-free* betting strategy for red versus black cards I outline in ["Kelly Can’t Fail"](https://win-vector.com/2024/12/19/kelly-cant-fail/). This gives some of the thrill of [card-counting](https://en.wikipedia.org/wiki/Card_counting), without risk or the labor of learning the rules of [blackjack](https://en.wikipedia.org/wiki/Blackjack).

In this note I dig in a bit deeper into the relation between drawing cards and flipping coins.

## Flipping Coins

Let's consider the simple exercise of flipping a fair coin that comes up "0" half the time and "1" half the time.  Represent our coin flips with the 0/1 valued random variables $X_{1}, \cdots, X_{n\_trials}$. 

Define the following:

  * The derived random variable $X = \sum_{i = 1}^{n\_trials} X_i$. $X$ is known to have the [Binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution), which tells us how likely each possible value of the sum $X$ tends to be.
  * The k-th [moment](https://en.wikipedia.org/wiki/Moment_(mathematics)): $\text{E}[X^k]$. This is the expected value of $X^k$ where the expectation is taken over independent repetitions of the entire experiment.  The first moment $\text{E}[X]$ is also called the expected value, and in our case is equal to $n\_trials / 2$.

How far off our observed $X$ tends to be from the ideal expected value $n\_trials / 2$ is encoded in the so-called higher moments (moments with `k > 1`). The smaller the higher moment (especially the value for `k = 2`) values are, the more tame and "typically near the expected value" the distribution tends to be.



### Coin Flipping Moments

For the coin flipping random variable we can calculate these moments exactly (using `sympy`) as follows.

In [1]:
from itertools import permutations, product
import numpy as np
from sympy import binomial, factorial, Integer
from sympy.functions.combinatorial.numbers import stirling

In [2]:
def k_th_binomial_moment(k, n_trials):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are i.i.d. probability 1/2 binomial random variables.
    Calculates direct expectation by summing each possible value times probability of that value.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    """
    k = Integer(k)
    n_trials = Integer(n_trials)
    assert k >= 1
    assert n_trials >= 1
    two = Integer(2)
    return np.sum([
        binomial(n_trials, sum_x)  # pick which selections are 1
        * two**(-sum_x)  # probability of these being simultaneously 1
        * two**(-(n_trials-sum_x))  # probability of the rest being simultaneously zero
        * sum_x**k  # value of sum implied by above choices
        for sum_x in range(0, n_trials + 1)  # possible values of sum
    ])

The above formula and algorithm is fairly typical of [enumerative combinatorics](https://en.wikipedia.org/wiki/Enumerative_combinatorics). One can look it up, derive it, read it off the so called moment generating function, or find it other ways. It is a learnable skill to decompose such counting problems into terms of powers, factorials, binomial coefficients, Stirling numbers, and other pre-solved notations. The first few moments seen for summing up 10 coin flips are as follows.

In [3]:
binomial_moments_10 = {
    k: float(k_th_binomial_moment(k=k, n_trials=10))
    for k in range(1, 5 + 1)
}

binomial_moments_10

{1: 5.0, 2: 27.5, 3: 162.5, 4: 1017.5, 5: 6687.5}

The first moment tells the expected sum (5) and the next one helps us bound the expected variation or variance.

The expected variance (or [2nd central moment](https://en.wikipedia.org/wiki/Central_moment)) is as follows.

In [4]:
binomial_variance = k_th_binomial_moment(k=2, n_trials=10) - k_th_binomial_moment(k=1, n_trials=10)**2

float(binomial_variance)

2.5

### Confirming the coin flip moments

We can confirm our coin flip moment calculation against a brute force calculation as follows.

In [5]:
def k_th_binomial_moment_bf(k, n_trials):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are i.i.d. probability 1/2 binomial random variables.
    Calculates by brute force enumeration of all possibilties.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    """
    num = 0
    den = 0
    for trial in product([0, 1], repeat=n_trials):
        num += np.sum(trial)**k
        den += 1
    return num/den

check_binomial_10 = {
    k: float(k_th_binomial_moment_bf(k=k, n_trials=10))
    for k in range(1, 5 + 1)
}
assert check_binomial_10 == binomial_moments_10

## Drawing Cards

We characterize drawing cards as follows. Consider a uniformly [permuted](https://en.wikipedia.org/wiki/Permutation) or shuffled deck with `s_cards` red cards (representing `0`s) and `s_cards` black cards (representing `1`s). Represent our card draws without replacement with the 0/1 valued random variables $Y_{1}, \cdots, Y_{n\_trials}$, where $Y_{i}$ is the `i`th card in this deck.

Define the following:

  * The derived random variable $Y = \sum_{i = 1}^{n\_trials} Y_i$. $Y$ has a new distribution called the [Hypergeometric distribution](https://en.wikipedia.org/wiki/Hypergeometric_distribution).
  * The k-th [moment](https://en.wikipedia.org/wiki/Moment_(mathematics)): $\text{E}[Y^k]$. This is the expected value of $Y^k$ where the expectation is taken over independent repetitions of the entire experiment.  The first moment $\text{E}[Y]$ is also called the expected value, and in our case is again equal to $n\_trials / 2$.

One can study $Y$ directly using facts about [hypergeometric distribution](https://en.wikipedia.org/wiki/Hypergeometric_distribution). Or (as is traditional) one can attempt to simplify and approximately study $Y$ in terms of a simpler distribution such as the [Poisson distribution](https://en.wikipedia.org/wiki/Poisson_distribution) (this is most appropriate for decks where `1`s are rare), or (even better) the binomial distribution (which is how $X$ is distributed). We will look at how $Y$ differs from the Binomial distribution, as that helps show what intuition needs to change in moving from coin flipping to card drawing.

### Card Drawing Moments

The moments for drawing cards (without replacement) can be calculated in a similar (though more complicated) manner.

In [6]:
def k_th_card_moment(k, n_trials, *, s_cards):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are drawn without replacement from a pool of s_cards 0's and s_cards 1's.
    Calculates direct expectation by summing each possible value times probability of that value.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    :param s_cards: draw from s_scards 0 cards plus s_cards 1 cards
    """
    k = Integer(k)
    n_trials = Integer(n_trials)
    s_cards = Integer(s_cards)
    assert k >= 1
    assert n_trials >= 1
    assert s_cards >= 1
    two = Integer(2)
    return np.sum([
        binomial(n_trials, sum_y)   # pick which selections are 1
        * (binomial(s_cards, sum_y) / binomial(two * s_cards, sum_y))  # probability of these being simultaneously 1
        * (binomial(s_cards, n_trials - sum_y) / binomial(two * s_cards - sum_y, n_trials - sum_y))  # probability of the rest being simultaneously zero
        * sum_y**k   # value of sum implied by above choices
        for sum_y in range(0, min(n_trials + 1, s_cards + 1))  # possible values of sum
    ])

This differs from the coin flip situation in that the moment depends on what size deck we are drawing from. For a large deck the results have to be very close to binomial statistics. For smaller decks we expect significant "card counting" opportunities.

For a very large deck the first few moments are nearly identical to the moments of the earlier binomial distribution.

In [7]:
{
    k: float(k_th_card_moment(k=k, n_trials=10, s_cards=1000))
    for k in range(1, 5 + 1)
}

{1: 5.0,
 2: 27.488744372186094,
 3: 162.33116558279139,
 4: 1015.6655693896022,
 5: 6669.7783042748215}

This is why casinos play blackjack from mixtures of many decks (and do not play deep into these decks): it quantitatively removes the advantage of card counting.

For a standard deck with 26 red cards (representing `0`s) and 26 black cards (representing `1`s) we have the following first few moments.

In [8]:
{
    k: float(k_th_card_moment(k=k, n_trials=10, s_cards=26))
    for k in range(1, 5 + 1)
}


{1: 5.0,
 2: 27.058823529411764,
 3: 155.88235294117646,
 4: 945.9663865546219,
 5: 6002.100840336135}

The first moment is the same as for coin flipping: we expect about 5 1s per draw. The second is a bit smaller for card drawing than for the coin flipping distribution. This is because, unlike coin flipping card draws tend to return towards the expected number as we draw more. This means observed counts being more concentrated near the ideal expected value, which in turn produce smaller values for the higher moments.

We can confirm the variance of the card drawing system is in fact less than the variance of the binomial system.

In [9]:
draw_variance = k_th_card_moment(k=2, n_trials=10, s_cards=26) - k_th_card_moment(k=1, n_trials=10, s_cards=26)**2
assert draw_variance < binomial_variance

float(draw_variance)

2.0588235294117645

### Confirming card draw momements

We can confirm our card draw moments are correct by comparing to simple brute force enumeration of a small deck.

In [10]:
draw_3_10_moments = {
    k: float(k_th_card_moment(k=k, n_trials=3, s_cards=3))
    for k in range(1, 5 + 1)
}

draw_3_10_moments

{1: 1.5, 2: 2.7, 3: 5.4, 4: 11.7, 5: 27.0}

In [11]:
def k_th_card_moment_bf(k, n_trials, *, s_cards):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are drawn without replacement from a pool of s_cards 0's and s_cards 1's.
    Calculates by brute force enumeration.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    :param s_cards: draw from s_scards 0 cards plus s_cards 1 cards
    """
    num = 0
    den = 0
    # deliberately inneficient draw of all (2 * s_cards)! permutations
    # (not factoring down to discernable ones)
    for perm in permutations([0] * s_cards + [1] * s_cards):
        draw = perm[0:n_trials]
        num += np.sum(draw)**k
        den += 1
    return num/den

check_3_10_moments = {
    k: float(k_th_card_moment_bf(k=k, n_trials=3, s_cards=3))
    for k in range(1, 5 + 1)
}
assert check_3_10_moments == draw_3_10_moments

## Comparing All The Moments

It turns out that all of the card drawing moments are no larger than the corresponding coin flipping moments!

First we empirically confirm this.

In [12]:
# confirm all higher card moments are smaller
for s_cards in range(1, 26):
    for n_trials in range(1, s_cards+1):
        assert k_th_binomial_moment(1, n_trials) > 0
        assert k_th_binomial_moment(1, n_trials) == k_th_card_moment(1, n_trials, s_cards=s_cards)
        for k in range(2, n_trials+1):
            assert k_th_binomial_moment(k, n_trials) > 0
            assert k_th_binomial_moment(k, n_trials) > k_th_card_moment(k, n_trials, s_cards=s_cards)

At this point we have a conjecture: all of the card drawing moments are no greater than the coin flipping moments. It turns out this is something we can in fact prove.

## Proving the Card Drawing Moments are no more Than Coin Flipping Moments

We want to show in general `k_th_binomial_moment(k, n_trials) >= k_th_card_moment(k, n_trials, s_cards=s_cards)`. However, each of these functions is a sum over slightly different terms. And it is not the case that each and every term of the `k_th_binomial_moment(k, n_trials)` sum is larger than the corresponding term of the `k_th_card_moment(k, n_trials, s_cards=s_cards)` sum (which would be an easy way to prove which sum is larger).  To complete our proof we introduce equivalent functions where the the coin flipping sum does in fact dominate the card drawing sum term by term, allowing us to prove which sum is larger.

We use an idea from [Zeilberger's "Symbolic Moment Calculus I"](https://sites.math.rutgers.edu/~zeilberg/mamarim/mamarimhtml/smcI.html).

Instead of calculating $\text{E}[X^k]$ as $\sum_{x} x^k \; \text{P}[X=x]$ we instead calculate it as $\sum_{i_1=1}^{n\_trials} \sum_{i_2=1}^{n\_trials} \cdots \sum_{i_k=1}^{n\_trials} \text{E}[X_{i_{1}} X_{i_{2}} \cdots X_{i_{n\_trials}}]$. This may seem strange, but it is exploiting the fact that $X = \sum_{i = 1}^{n\_trials} X_i$, and then expanding the product of sums into a nested sum of products.

In this form our two moment calculations are as follows.

In [13]:
def k_th_binomial_moment_sm(k, n_trials):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are i.i.d. probability 1/2 binomial random variables.
    Calculated by expansion of sum into 0/1 product terms ala Zeilberger Symbolic Moment Calculus.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    """
    k = Integer(k)
    n_trials = Integer(n_trials)
    assert k >= 1
    assert n_trials >= 1
    two = Integer(2)
    return np.sum([
        binomial(n_trials, support) *   # number of ways to select with distinct values from 1 ... n_trials (product we expect to be non-zero as we picked only non-zero random variables)
        stirling(k, support) * factorial(support)   # number i_{1} ... i_{n_trials} assignments with support distinct values (which indices picked the non-zero random variables)
        * two**(-support)   # probability of all selected terms being 1
        for support in range(1, min(k + 1, n_trials + 1))  # number of distinct indicator indices selected in product term
    ])

The more complicated sum is part of the magic of the symbolic moment method. If one can (either by human work or computer algebra) work out how often the product $X_{i_{1}} X_{i_{2}} \cdots X_{i_{n\_trials}}$ is non-zero, then one has the value of the `k`th moment. The difficulty in the calculation is the accounting for the $i_{j}$ picking the same random variables. We introduced [support](https://en.wikipedia.org/wiki/Support_(mathematics)) as the size of the set of random variable picked to help organize counting these cases.

If we have gotten all the mathematics and calculation right then we would have: `k_th_binomial_moment(k, n_trials) = k_th_binomial_moment_sm(k, n_trials)` in general. Let's confirm this is indeed the case.

In [14]:
# confirm agrees with earlier definition
for k in range(1, 27):
    for n_trials in range(1, 27):
        assert k_th_binomial_moment(k, n_trials) == k_th_binomial_moment_sm(k, n_trials)

The claim `k_th_binomial_moment(k, n_trials) = k_th_binomial_moment_sm(k, n_trials)` in general (for integer `n_trials > 0`, `k > 0`, ...) can be written in math notation (instead of Python) as the following lemma or claim.

$$
\begin{align*}
k\_th\_binomial\_moment(k, n\_trials) &= \sum_{x = 0}^{n\_trials} \binom{n\_trials}{x} 2^{-x} 2^{-(n\_trials - x)} x^{k} \\
k\_th\_binomial\_moment\_sm(k, n\_trials) &= \sum_{s = 1}^{min(k, n\_trials)} \binom{n\_trials}{s} S(k, s) (s !) 2^{-s}
\end{align*}
$$

So:

$$
\sum_{x = 0}^{n\_trials} \binom{n\_trials}{x} 2^{-x} 2^{-(n\_trials - x)} x^{k}
=
\sum_{s = 1}^{min(k, n\_trials)} \binom{n\_trials}{s} S(k, s) (s !) 2^{-s}
$$

This is considered proven under the ["counting two ways"](https://en.wikipedia.org/wiki/Double_counting_(proof_technique)) methodology: if they are both correctly calculating the same thing then they must be equal. Re-factoring combinatorial counting problems routinely establishes chains of "this sum turns out to equal this seemingly different sum" identities. This is in fact the minimal amount of mathematical discovery required to effectively calculate combinations. 

The desired difference in the calculations: the older method needs to directly track the probabilities of seeing `0`s and `1`s, and the newer (`*_sm()`) method directly tracks only the probabilities of seeing `1`s. This is going to make switching between binomial (coin flip) and card draw distributions easier, as it will mean we replace less of the formula or code when switching from one to another. This will be made explicit as we work.

We re-write our $\text{E}[Y^k]$ calculation similarly to get.

In [15]:
def k_th_card_moment_sm(k, n_trials, *, s_cards):
    """
    Compute expected value of (sum_{i = 1 ... n_trials} x_i)**k where x_i are drawn without replacement from a pool of s_cards 0's and s_cards 1's.
    Calculated by expansion of sum into 0/1 product terms ala Zeilberger Symbolic Moment Calculus.

    :param k: moment to compute
    :param n_trials: number of trials (flip or draw events) in the sum
    :param s_cards: draw from s_scards 0 cards plus s_cards 1 cards
    """
    k = Integer(k)
    n_trials = Integer(n_trials)
    s_cards = Integer(s_cards)
    assert k >= 1
    assert n_trials >= 1
    assert s_cards >= 1
    two = Integer(2)
    return np.sum([
        binomial(n_trials, support) *   # number of ways to select with distinct values from 1 ... n_trials (product we expect to be non-zero as we picked only non-zero random variables)
        stirling(k, support) * factorial(support)   # number i_{1} ... i_{n_trials} assignments with support distinct values (which indices picked the non-zero random variables)
        * (binomial(s_cards, support) / binomial(two * s_cards, support))  # probability of these being simultaneously 1
        for support in range(1, min(k + 1, n_trials + 1))  # number of distinct indicator indices selected in product term
    ])

The claim `k_th_card_moment(k, n_trials, s_cards=s_cards) = k_th_card_moment_sm(k, n_trials, s_cards=s_cards)` in general can be written in math notation (instead of Python) as the following lemma.


$$
\begin{align*}
k\_th\_card\_moment(k, n\_trials, s\_cards=s\_cards) &= 
\sum_{y = 0}^{n\_trials} \binom{n\_trials}{y} \frac{ \binom{s\_cards}{y} }{ \binom{2 \; s\_cards}{y} } \frac{ \binom{s\_cards}{n\_trials - y} }{ \binom{2 \; s\_cards - y}{n\_trials - y} } y^{k} \\
k\_th\_card\_moment\_sm(k, n\_trials, s\_cards=s\_cards) &= 
\sum_{s = 1}^{min(k, n\_trials)} \binom{n\_trials}{s} S(k, s) (s !) \frac{ \binom{s\_cards}{s} }{ \binom{2 \; s\_cards}{s} } 
\end{align*}
$$

So:

$$
\sum_{y = 0}^{n\_trials} \binom{n\_trials}{y} \frac{ \binom{s\_cards}{y} }{ \binom{2 \; s\_cards}{y} } \frac{ \binom{s\_cards}{n\_trials - y} }{ \binom{2 \; s\_cards - y}{n\_trials - y} } y^{k}
=
\sum_{s = 1}^{min(k, n\_trials)} \binom{n\_trials}{s} S(k, s) (s !) \frac{ \binom{s\_cards}{s} }{ \binom{2 \; s\_cards}{s} }
$$

Let's numerically confirm this claim.


In [16]:
# confirm agrees with earlier definition
for s_cards in range(1, 26):
    for n_trials in range(1, s_cards+1):
        for k in range(1, s_cards+1):
            assert k_th_card_moment(k, n_trials, s_cards=s_cards) == k_th_card_moment_sm(k, n_trials, s_cards=s_cards)

Re-factoring the calculation (and knowing to use the `stirling()` function) are specialized, but standard methods in combinatorics.

What we want to call out is: the two new calculations (the `*_sm()` functions) differ only in that:

  * The coin flipping calculation has a `2**(-support)` term estimating the probability of seeing 1s in the expected positions.
  * The card drawing calculation has a `(binomial(s_cards, support) / binomial(2 * s_cards, support))` term estimating the probability of seeing 1s in the expected positions.

This difference is simpler than the difference between the original (non `*_sm()` functions) functions (which had to account for the probabilities of not just the 1s, but the 0s and the 1s).

With some work one can show that `2**(-support) >= (binomial(s_cards, support) / binomial(2 * s_cards, support))`. The algebra isn't really worth writing down, so we just check an example value.



In [17]:
support = 3
s_cards = 25

assert 2**(-support) >= (binomial(s_cards, support) / binomial(2 * s_cards, support))

(float(2**(-support)), 
 float(binomial(s_cards, support) / binomial(2 * s_cards, support)))

(0.125, 0.11734693877551021)

Let's accept `2**(-support) >= (binomial(s_cards, support) / binomial(2 * s_cards, support))` in general. Then each term in the `k_th_binomial_moment_sm()` sum is in fact no smaller than the corresponding term in the `k_th_card_moment_sm()` sum. This in term completes the proof that the binomial moments are no smaller than the card drawing moments.

We've confirmed and proven the statement, but let's look at it to be triply sure. One really has to look to be sure.

In [18]:
for k in range(1, 5):
    print((
        float(k_th_binomial_moment_sm(k=k, n_trials=10)), 
        float(k_th_card_moment_sm(k=k, n_trials=10, s_cards=s_cards))
    ))

(5.0, 5.0)
(27.5, 27.040816326530614)
(162.5, 155.6122448979592)
(1017.5, 943.0633955709943)


And we see the second term is never larger.

## Conclusion

Card drawing is more tightly concentrated than coin flipping. This is why card counting works. When one has drawn too many red cards, one knows more black cards remain to be draw. This is in contrast to coin flipping, where streaks do not in fact have a propensity to undo themselves. In card counting one knows the expected rates and bets on reversions back to them.